In [1]:
import pandas as pd
import matplotlib.pyplot as plot
import numpy as np

In [2]:
members = pd.read_csv('../data/NSS_GreatCare_Members.csv', low_memory=False)
inpatient = pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv', low_memory=False)
pcp_visits = pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv', low_memory=False)

In [ ]:
members.info()

In [ ]:
members1 = members[['Patient ID', 'Primary Plan ID']]

In [ ]:
members1.head()

In [4]:
#running this before creating the new inpatient df
inpatient['Encounter Start Date'] = pd.to_datetime(inpatient['Encounter Start Date'], format='%Y-%m-%d')
inpatient['Encounter End Date'] = pd.to_datetime(inpatient['Encounter End Date'], format='%Y-%m-%d')

In [6]:
inpatient1=inpatient[['Encounter ID', 'Patient ID', 'Encounter Start Date', 'Encounter End Date']]

In [7]:
inpatient1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Encounter ID          6213 non-null   object        
 1   Patient ID            6213 non-null   object        
 2   Encounter Start Date  6213 non-null   datetime64[ns]
 3   Encounter End Date    6213 non-null   datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 194.3+ KB


In [ ]:
pcp_visits.info()

In [ ]:
pcp_visits1.head()

In [ ]:
#to show all the rows:
#pd.set_option('display.max_columns', None)

# Question 1
### What is our 30-day all cause readmission rate across hospitals our members are admitted to?

In [ ]:
#if there is a second admission date, subtract the previous visit's end date .
#Find where the time is < 30 days

In [8]:
#first step is to sort the inpatient1 df
inpatient1 = inpatient1.sort_values(['Patient ID',"Encounter Start Date", "Encounter End Date"], ascending = (False, True, True))

In [9]:
#THIS IS GOLD KEEP IT.  Creates columns for us to do math on and then identify rows where th patient has been admitted twice
inpatient1['Previous End Date'] = (inpatient1.groupby('Patient ID')['Encounter End Date'].shift(1))
inpatient1['Days Since Last Visit'] = inpatient1['Encounter Start Date'] - inpatient1['Previous End Date']
inpatient1 = inpatient1[inpatient1['Days Since Last Visit'].notnull()]




In [10]:
inpatient1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 6211 to 7
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   Encounter ID           1512 non-null   object         
 1   Patient ID             1512 non-null   object         
 2   Encounter Start Date   1512 non-null   datetime64[ns] 
 3   Encounter End Date     1512 non-null   datetime64[ns] 
 4   Previous End Date      1512 non-null   datetime64[ns] 
 5   Days Since Last Visit  1512 non-null   timedelta64[ns]
dtypes: datetime64[ns](3), object(2), timedelta64[ns](1)
memory usage: 82.7+ KB


In [15]:
#Creates new dataframe where time since last visit is less than 30 days
inpatient30d = inpatient1[inpatient1['Days Since Last Visit'] <= '30 days']
inpatient30d.info()
#755 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755 entries, 6211 to 44
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   Encounter ID           755 non-null    object         
 1   Patient ID             755 non-null    object         
 2   Encounter Start Date   755 non-null    datetime64[ns] 
 3   Encounter End Date     755 non-null    datetime64[ns] 
 4   Previous End Date      755 non-null    datetime64[ns] 
 5   Days Since Last Visit  755 non-null    timedelta64[ns]
dtypes: datetime64[ns](3), object(2), timedelta64[ns](1)
memory usage: 41.3+ KB


In [19]:
#Getting the readmission within 30 days rate:
inpatient['Patient ID'].nunique() #4701
inpatient30d['Patient ID'].nunique() #479

479

In [21]:
rate = inpatient30d['Patient ID'].nunique() / inpatient['Patient ID'].nunique()
print(rate) #10.18932%

0.10189321420974261


# Question2:
### What percent of patients have a PCP visit within a week of discharge?

In [31]:
#From Brenda:
#make df and merge
ad_cut = inpatient[['Patient ID','Encounter End Date']]
pcp_cut = pcp_visits[['Patient ID','Encounter Date']]
together = ad_cut.merge(pcp_cut, left_on ='Patient ID', right_on = 'Patient ID',how='left', indicator = False)
#sort
together = together.sort_values(['Patient ID',"Encounter Date", "Encounter End Date"], ascending = (False, True, True))
#change to datetime
together['Encounter Date'] = pd.to_datetime(together['Encounter Date'], format='%Y-%m-%d')
#sub encounter date from end dates
together['Days Since Last Visit'] = together['Encounter Date'] - together['Encounter End Date']
two_top = together[together['Days Since Last Visit'] <= '7 days']
two_top = two_top[two_top['Days Since Last Visit'] >= '0 days']
two_top.rename(columns={"Encounter End Date": "Inpatient End Date"}, inplace = True)
two_top

,Patient ID,Inpatient End Date,Encounter Date,Days Since Last Visit
44074,DS1|9981,2014-08-10,2014-08-11,1 days
44077,DS1|9981,2014-08-10,2014-08-12,2 days
44071,DS1|9979,2015-02-11,2015-02-15,4 days
44067,DS1|9978,2015-09-05,2015-09-11,6 days
44051,DS1|9945,2013-02-11,2013-02-14,3 days
...,...,...,...,...
238,DS1|10303,2012-06-24,2012-06-28,4 days
231,DS1|10303,2012-06-24,2012-06-30,6 days
50,DS1|10101,2012-12-09,2012-12-12,3 days
28,DS1|10073,2012-06-21,2012-06-27,6 days
